<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://www.islas.org.mx//">     
           <img 
               src="assets/geci_20_anos.png" 
               style="width:200px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Erradicación de gatos en Socorro</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2> Las cuatro gráficas</h2></div>

In [1]:
import pandas as pd
import numpy as np
import gatos as dmc
import matplotlib.pyplot as plt
import bokeh.plotting as bp
import bokeh.models as bm
import bokeh.io as bi
from bokeh.layouts import row
from math import ceil

ModuleNotFoundError: No module named 'gatos'

## Objetivo
Generar cuatro gráficas:
1. Serie de tiempo capturas por esfuerzo
1. Serie de tiempo capturas acumuladas
1. Capturas acumuladas en función del esfuerzo acumulado
1. Capturas por esfuerzo en función del esfuerzo aculumado

## Lectura de datos

### Dirección de las carpetas

In [ ]:
carpeta_DP = "../inst/extdata/erradicaciones-mamiferos/"
carpeta_resultados = "../resultados/"

### Nombre del recurso

In [ ]:
nombres = 'captura_gatos.csv'

### Lectura de datos y metadatos

In [ ]:
GatosConMetadatos = dmc.DataAndMetadataCats(carpeta_DP, nombres)

In [ ]:
tiempo = GatosConMetadatos.glosario['tiempo']
esfuerzo = GatosConMetadatos.glosario['esfuerzo']
capturas = GatosConMetadatos.glosario['capturas']

## Limpia de los datos

### Caso particular

In [ ]:
datos_gatos = GatosConMetadatos.getData()
sum(datos_gatos.esfuerzo)

## Graficado
### Serie de tiempo captura por esfuerzo

In [ ]:
captura_por_esfuerzo = datos_gatos[capturas].values / datos_gatos[esfuerzo].values

In [ ]:
plt.figure(figsize = (8, 4))
plt.plot(datos_gatos.tiempo_listo, captura_por_esfuerzo, "o")
plt.ylabel("Catch per effort unit (cats/trap$\cdot$night)")
plt.xlabel(" ")
eje = plt.gca()
eje.spines["top"].set_visible(False)
eje.spines["right"].set_visible(False)
plt.ylim([0.0, 0.1])
plt.savefig("../resultados/catch-per-unit-effort_time-serie.png")

### Serie de tiempo captura acumudalas

In [ ]:
captura_acumuladas = datos_gatos.capturas.cumsum()

In [ ]:
plt.figure(figsize = (8, 4))
plt.plot(datos_gatos.tiempo_listo, captura_acumuladas, "o")
plt.ylabel("Accumulated catch (cats)")
plt.xlabel(" ")
plt.yticks(np.arange(50, 651, step=100))
eje = plt.gca()
eje.spines["top"].set_visible(False)
eje.spines["right"].set_visible(False)
plt.ylim([0.0, 650])
plt.savefig("../resultados/accumulated-catch_time-serie.png")

### Captura acumudalas en función del esfuerzo acumulado

In [ ]:
esfuerzo_acumuladas = datos_gatos.esfuerzo.cumsum()

In [ ]:
plt.figure(figsize = (8, 4))
plt.plot(esfuerzo_acumuladas, captura_acumuladas, "o")
plt.ylabel("Accumulated catch (cats)")
plt.xlabel("Accumulated effort (trap-night)")
plt.yticks(np.arange(50, 651, step=100))
eje = plt.gca()
eje.spines["top"].set_visible(False)
eje.spines["right"].set_visible(False)
plt.ylim([0.0, 651])
plt.savefig("../resultados/accumulated-catch_accumulated-effort.png")

In [ ]:
bi.curdoc().clear()
bp.reset_output()

In [ ]:
fuente = bp.ColumnDataSource({'x': esfuerzo_acumuladas, 'ca': pd.Series(captura_acumuladas), 'e': pd.Series(captura_por_esfuerzo)})
encima_p1 = bm.HoverTool(tooltips=[
    ("Esfuerzo acumulado", "@x"),
    ("Capturas acumuladas", "@ca")],
    mode='mouse')
encima_p2 = bm.HoverTool(tooltips=[
    ("Esfuerzo acumulado", "@x"),
    ("Capturas por esfuerzo", "@e")],
    mode='mouse')
herramientas = "box_select,lasso_select,pan,box_zoom,wheel_zoom,reset,save"

In [ ]:
titulo_figura = "Capturas acumuladas vs esfuerzo acumulado"
limite_y = ceil(max(captura_acumuladas)/100) * 100
opciones_grafica = dict(title=titulo_figura, x_axis_label='Esfuerzo acumulado (trampas * noche)', \
                        y_axis_label='Capturas acumuladas (individuos)', y_range=(0,limite_y),\
                        tools=[encima_p1, herramientas])

In [ ]:
p1 = bp.figure(**opciones_grafica)
p1.title.text_font = "verdana"
p1.yaxis.axis_label_text_font = "verdana"
p1.yaxis.axis_label_text_font_style = "normal"
p1.xaxis.axis_label_text_font = "verdana"
p1.xaxis.axis_label_text_font_style = "normal"
p1.circle('x', 'ca', size=10, source=fuente)
p1.grid.visible = False

### Captura por esfuerzo en función del esfuerzo acumulado

In [ ]:
plt.figure(figsize = (8, 4))
plt.plot(esfuerzo_acumuladas, captura_por_esfuerzo, "o")
plt.ylabel("Catch per effort unit (cats/trap$\cdot$night)")
plt.xlabel("Accumulated effort (trap$\cdot$night)")
eje = plt.gca()
eje.spines["top"].set_visible(False)
eje.spines["right"].set_visible(False)
plt.ylim([0.0, 0.1])
plt.savefig("../resultados/catch-per-unit-effort_accumulated-effort.png")

In [ ]:
titulo_figura = "Capturas por esfuerzo vs esfuerzo acumulado"
limite_y = 0.1
opciones_grafica = dict(title=titulo_figura, x_axis_label='Esfuerzo acumulado (trampas * noche)', \
                        y_axis_label='Capturas por esfuerzo (individuos/(trampas * noche))', y_range=(0,limite_y),\
                        tools=[encima_p2, herramientas])

In [ ]:
p2 = bp.figure(**opciones_grafica)
p2.title.text_font = "verdana"
p2.yaxis.axis_label_text_font = "verdana"
p2.yaxis.axis_label_text_font_style = "normal"
p2.xaxis.axis_label_text_font = "verdana"
p2.xaxis.axis_label_text_font_style = "normal"
p2.circle('x', 'e', size=10, source=fuente)
p2.grid.visible = False

In [ ]:
p = bp.gridplot([[p1, p2]])
bp.output_file('../resultados/catch-effort.html')
bp.show(p)

In [ ]:
ceil(max(captura_acumuladas)/100) * 100